# Introduction to CREsted

In this introductory notebook, we will train a topic classification model and inspect the results.

In [1]:
import crested
import os

2024-06-13 15:20:24.115927: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 15:20:24.198815: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 15:20:25.614317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-13 15:20:32.714606: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


We can use the {func}`~crested.import_peaks` function to import data into an {class}`anndata.AnnData` object,
with the imported topics as the `AnnData.obs` and the consensus peak regions as the `AnnData.var`.  
Below we assume that you have environment variables set to your data paths, but you can simply provide the strings.

In [7]:
adata = crested.import_topics(
    topics_folder="/staging/leuven/stg_00002/lcb/lmahieu/projects/DeepTopic/biccn_test/otsu",
    regions_file="/staging/leuven/stg_00002/lcb/lmahieu/projects/DeepTopic/biccn_test/consensus_peaks_bicnn.bed",
    compress=True,
    # topics_subset=["topic_1", "topic_2"], # optional subset of topics to import
)
adata

FileNotFoundError: Directory '/staging/leuven/stg_00002/lcb/lmahieu/projects/DeepTopic/biccn_test/otsu' not found

To train a model, we'll need to add a *split* column to our dataset, which we can do using `crested.pp.train_val_test_split()`.  
We can add a `random_state` to ensure the data will be split in the same manner in the future when `shuffle=True`(default).

In [3]:
# We can split randomly on the regions
crested.pp.train_val_test_split(
    adata, type="random", val_size=0.1, test_size=0.1, random_state=42
)

# Or, choose the chromosomes for the validation and test sets
# enhai.pp.train_val_test_split(
#     adata, type="chr", chr_val=["chr4", "chrX"], chr_test=["chr2", "chr3"]
# )

print(adata.var["split"].value_counts())
adata.var

split
train    437593
test      54700
val       54700
Name: count, dtype: int64


,n_topics,chr,start,end,split
region,,,,,
chr1:3094805-3095305,5,chr1,3094805,3095305,train
chr1:3095470-3095970,0,chr1,3095470,3095970,train
chr1:3112174-3112674,1,chr1,3112174,3112674,test
chr1:3113534-3114034,2,chr1,3113534,3114034,train
chr1:3119746-3120246,8,chr1,3119746,3120246,train
...,...,...,...,...,...
chrX:169879313-169879813,3,chrX,169879313,169879813,train
chrX:169880181-169880681,0,chrX,169880181,169880681,train
chrX:169925477-169925977,1,chrX,169925477,169925977,train


## Train